# Exploring different data files to see what they look like

In [1]:
# Standard Library
import warnings
import concurrent
from concurrent.futures import ThreadPoolExecutor
import math

# Third-party Libraries
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# from IPython.display import set_matplotlib_formats
from scipy.stats import mode
from scipy.spatial import cKDTree
from geopy.distance import geodesic


# Rasterio
import rasterio
from rasterio.features import shapes
from rasterio.mask import mask

# Shapely
from shapely.geometry import box
from shapely.geometry import Point

# Linear Models
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects

import os

# Ignore warnings
warnings.filterwarnings('ignore')

# set_matplotlib_formats('retina')

In [5]:
boundary_shp_file = "/gypsum/eguide/projects/ce8760/locations/rwanda_boundary/RWA_adm0.shp"

In [6]:
gpd.read_file(boundary_shp_file).crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Helper functions to clip tif files and process data into administrative regions

In [7]:
def get_admin_boundary(admin_level: str) -> str:
    """
    This function takes in the admin_level string and returns the file path to the shape file

    Input:
        - admin_level: A string indicating the admin level (district, sector, cell, village, or boundary)

    Returns:
        - String representing the shape file path
    """

    admin_paths = {
        "district": "/gypsum/eguide/projects/ce8760/locations/district/District.shp",
        "sector": "/gypsum/eguide/projects/ce8760/locations/sector/Sector.shp",
        "cell": "/gypsum/eguide/projects/ce8760/locations/cell/Cell.shp",
        "village": "/gypsum/eguide/projects/ce8760/locations/villages/Village.shp",
        "boundary": "/gypsum/eguide/projects/ce8760/locations/rwanda_boundary/RWA_adm0.shp",
    }

    admin_ids = {
        "district": "Dist_ID",
        "sector": "Sect_ID",
        "cell": "Cell_ID",
        "village": "Village_ID",
        "boundary": None,
    }

    # Use the provided admin_level to directly access the file path from the dictionary
    admin_path = admin_paths.get(admin_level.lower())
    admin_id = admin_ids.get(admin_level.lower())

    if admin_path is None:
        raise ValueError(f"Invalid admin_level: {admin_level}")

    return admin_path, admin_id

In [8]:
def clip_tif_chunk(chunk):
    tif_file, chunk_bbox = chunk

    with rasterio.open(tif_file) as src:
        # Create a GeoDataFrame with the chunk bounding box
        bbox_gdf = gpd.GeoDataFrame(geometry=[box(*chunk_bbox)], crs=src.crs)

        # Open the boundary shapefile
        boundary_gdf = gpd.read_file(get_admin_boundary("boundary")[0])
        boundary_gdf = boundary_gdf.to_crs(src.crs)

        # Intersect the bounding box with the boundary shapefile
        intersection = gpd.overlay(boundary_gdf, bbox_gdf, how='intersection')

        # Check if the intersection is empty
        if intersection.empty:
            print(f"No intersection for chunk: {chunk}")
            return gpd.GeoDataFrame()

        # Clip the raster to the intersection geometry
        clipped, transform = mask(src, shapes=intersection.geometry, crop=True)

        # Create a GeoDataFrame directly from the clipped raster
        shapes_gen = rasterio.features.shapes(clipped, transform=transform)
        features = [{'geometry': geometry, 'properties': {'pixel_value': value}}
                    for (geometry, value) in shapes_gen]
        gdf_clipped = gpd.GeoDataFrame.from_features(features, crs=src.crs)

        gdf_clipped = gdf_clipped.to_crs(("EPSG:4326"))

    return gdf_clipped


def parallel_clip_large_tif(tif_file, num_chunks=4):
    with rasterio.open(tif_file) as src:
        xmin, ymin, xmax, ymax = src.bounds
        chunk_width = (xmax - xmin) / num_chunks

        # Define chunks based on bounding box
        chunks = [(tif_file, (xmin + i * chunk_width, ymin, xmin + (i + 1) * chunk_width, ymax))
                  for i in range(num_chunks)]

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(clip_tif_chunk, chunks))

    # Merge the results if needed
    print(results)
    final_result = gpd.GeoDataFrame(pd.concat(results, ignore_index=True), crs='EPSG:4326')

    return final_result

# # Example usage:
# tif_file = '/path/to/large.tif'
# clipped_data = parallel_clip_large_tif(tif_file)


In [9]:
def clip_tif_chunk(chunk):
    tif_file, chunk_bbox = chunk

    with rasterio.open(tif_file) as src:
        # Create a GeoDataFrame with the chunk bounding box
        bbox_gdf = gpd.GeoDataFrame(geometry=[box(*chunk_bbox)], crs=src.crs)

        # Open the boundary shapefile
        boundary_gdf = gpd.read_file(get_admin_boundary("boundary")[0])
        boundary_gdf = boundary_gdf.to_crs(src.crs)

        # Intersect the bounding box with the boundary shapefile
        intersection = gpd.overlay(boundary_gdf, bbox_gdf, how='intersection')

        # Check if the intersection is empty
        if intersection.empty:
            print(f"No intersection for chunk: {chunk}")
            return gpd.GeoDataFrame()

        # Clip the raster to the intersection geometry
        clipped, transform = mask(src, shapes=intersection.geometry, crop=True)

        # Create a GeoDataFrame directly from the clipped raster
        shapes_gen = rasterio.features.shapes(clipped, transform=transform)
        features = [{'geometry': geometry, 'properties': {'pixel_value': value}}
                    for (geometry, value) in shapes_gen]
        gdf_clipped = gpd.GeoDataFrame.from_features(features, crs=src.crs)

        gdf_clipped = gdf_clipped.to_crs(("EPSG:4326"))

    return gdf_clipped

def parallel_clip_large_tif(tif_file, num_chunks=4):
    with rasterio.open(tif_file) as src:
        xmin, ymin, xmax, ymax = src.bounds
        chunk_width = (xmax - xmin) / num_chunks

        # Define chunks based on bounding box
        chunks = [(tif_file, (xmin + i * chunk_width, ymin, xmin + (i + 1) * chunk_width, ymax))
                  for i in range(num_chunks)]

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Filter out chunks with no intersection
        valid_chunks = [(tif, bbox) for tif, bbox in chunks if has_intersection(tif, bbox)]
        results = list(executor.map(clip_tif_chunk, valid_chunks))

    # Merge the results if needed
    final_result = gpd.GeoDataFrame(pd.concat(results, ignore_index=True), crs='EPSG:4326')

    return final_result

def has_intersection(tif_file, bbox):
    with rasterio.open(tif_file) as src:
        bbox_gdf = gpd.GeoDataFrame(geometry=[box(*bbox)], crs=src.crs)
        boundary_gdf = gpd.read_file(get_admin_boundary("boundary")[0])
        boundary_gdf = boundary_gdf.to_crs(src.crs)
        intersection = gpd.overlay(boundary_gdf, bbox_gdf, how='intersection')
        return not intersection.empty

### Helper function for computing metrics in administrative boundaries

In [11]:
def compute_administrative_metric(gdf: gpd.GeoDataFrame, admin_level: str, summary_method: str = "median") -> gpd.GeoDataFrame:
    """
    This function takes a GeoPandas DataFrame containing an index spread across pixels, 
    and aggregates it to a higher administrative level such as sector, cell, or village.

    Args:
        gdf: GeoPandas GeoDataFrame containing the index values.
        admin_level: A string indicating the administrative level ('sector', 'cell', or 'village').

    Returns:
        GeoDataFrame with aggregated values based on the specified summary method.
    """

    # Define a helper function for mode calculation
    def calc_mode(x):
        # Calculate mode using scipy, but ensure it's treated as array-like
        modes = mode(x, keepdims=True)  # `keepdims` ensures consistency across versions
        if modes.count.size == 0 or modes.mode.size == 0:
            return None  # or np.nan or some fallback value
        
        mode_value = modes.mode[0]
        
        if mode_value == 10:
            # Exclude 10 and try again
            new_modes = mode(x[x != mode_value], keepdims=True)
            return new_modes.mode[0] if new_modes.count.size > 0 else mode_value
        
        return mode_value

    # Get the file path and identifier for the specified admin level
    admin_path, admin_id = get_admin_boundary(admin_level=admin_level)

    # Read the admin shapefile
    admin_shp = gpd.read_file(admin_path)

    # Set CRS of admin_shp to EPSG:4326
    admin_shp = admin_shp.to_crs("EPSG:4326")

    # Use GeoPandas sjoin for spatial join by intersection
    index_summary = gpd.sjoin(admin_shp, gdf, how="inner", predicate='intersects')

    # Define aggregation method dynamically based on input
    if summary_method == "mode":
        agg_method = calc_mode
    else:
        # Use the string method directly for "median" or "mean"
        agg_method = summary_method

    # Aggregate data and retain the first geometry in case of multiple intersections
    index_summary = index_summary.groupby([admin_id]).agg({
        "pixel_value": agg_method,
        "geometry": "first" 
    }).reset_index()

    # Create a new GeoDataFrame with the necessary columns and CRS
    index_summary = gpd.GeoDataFrame(index_summary[[admin_id, "pixel_value", "geometry"]],
                                     geometry="geometry", crs="EPSG:4326")

    return index_summary


### Ploting functions

In [12]:
def plot_geopandas(gdf, column, figsize=(10, 8), cmap="Reds", 
                             colorbar_title="Colorbar Title", plot_title="plot title",
                             boundary_shp_file="/gypsum/eguide/projects/ce8760/locations/rwanda_boundary/RWA_adm0.shp"):
    """
    This function plots a geopandas shapefile to show the disdribution of an attribute

    Parameters:
        - gdf: GeoPandas GeoDataFrame to be plotted.
        - column: Name of the column in the GeoDataFrame to be used for coloring.
        - figsize: Tuple specifying the figure size (default is (10, 8)).
        - cmap: Colormap to be used for coloring (default is "viridis").

    Returns:
        - None
    """
    # Set the style using seaborn
    sns.set_theme(style="ticks", palette="pastel")
    sns.set(font_scale=0.7, rc={"figure.dpi":300, 'savefig.dpi':300})
    # sns.set(font="Verdana", font_scale=0.7, rc={"figure.dpi":300, 'savefig.dpi':300})

    # Load the boundary shapefile and reproject to EPSG:4326
    boundary_gdf = gpd.read_file(boundary_shp_file).to_crs("EPSG:4326")
    
    # Reproject the main GeoDataFrame to EPSG:4326
    gdf = gdf.to_crs("EPSG:4326")

    # Create the plot
    fig, ax = plt.subplots(figsize=figsize)

    # Plot the boundary line (adjust linewidth and color)
    boundary_gdf.boundary.plot(ax=ax, linewidth=0.5, color="black")

    # Plot the GeoDataFrame with the specified column for coloring
    gdf.plot(column=column, cmap=cmap, linewidth=0.2, ax=ax, edgecolor="0.5")

    # Remove axis labels and boundary box
    ax.set_axis_off()

    # Show the colormap legend on the side with 4 decimal places and without the boundary box
    cax = fig.add_axes([0.92, 0.2, 0.05, 0.2])  # Adjust the position as needed
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=gdf[column].min(), vmax=gdf[column].max()))
    sm._A = []  # fake up the array of the scalar mappable
    cbar = fig.colorbar(sm, cax=cax, format="%.1f", drawedges=False)  # format="%.4f" for 4 decimal places

    # Set the title for the colorbar
    cbar.set_label(colorbar_title, rotation=270, labelpad=15)

    # Add title to the bottom of the plot
    # plt.suptitle(plot_title, x=0.5, y=0.05, fontsize=12, fontname="Verdana",
    #              ha='center', va='bottom')
    
    plt.suptitle(plot_title, x=0.5, y=0.05, fontsize=12, ha='center', va='bottom')

    # Show the plot
    plt.show()


In [13]:
def plot_geopandas_grid(gdf, years, figsize=(12, 12), cmap="Reds", colorbar_title="Colorbar Title", 
                        boundary_shp_file="/gypsum/eguide/projects/ce8760/locations/rwanda_boundary/RWA_adm0.shp", filename=None):
    """
    This function plots a geopandas GeoDataFrame on a 3x3 grid, showing the distribution of attributes for the specified years.

    Parameters:
        - gdf: GeoPandas GeoDataFrame with columns ['administrative_boundary', '2012', '2013', ..., '2020', 'geometry'].
        - years: List of years to be plotted (e.g., ['2013', '2014', ..., '2019']).
        - figsize: Tuple specifying the figure size (default is (15, 15)).
        - cmap: Colormap to be used for coloring (default is "Reds").
        - colorbar_title: Title for the colorbar.
        - boundary_shp_file: Path to the shapefile containing the boundary.
        - filename: Optional filename to save the plot as an SVG file.

    Returns:
        - None
    """
    # Load the boundary shapefile and reproject to EPSG:4326
    boundary_gdf = gpd.read_file(boundary_shp_file).to_crs("EPSG:4326")
    
    # Reproject the main GeoDataFrame to EPSG:4326
    gdf = gdf.to_crs("EPSG:4326")

    # Set the style using seaborn
    sns.set_theme(style="ticks", palette="pastel")
    # sns.set(font="Verdana", font_scale=0.7, rc={"figure.dpi": 300, 'savefig.dpi': 300})
    sns.set(font_scale=0.7, rc={"figure.dpi": 300, 'savefig.dpi': 300})

    # Create the subplots
    fig, axes = plt.subplots(3, 3, figsize=figsize, sharex=True, sharey=True)
    
    # Plot each year on the grid
    for i, year in enumerate(years):
        row, col = divmod(i, 3)
        ax = axes[row, col]

        # Plot the boundary line (adjust linewidth and color)
        boundary_gdf.boundary.plot(ax=ax, linewidth=0.5, color="black")

        # Plot the GeoDataFrame with the specified column for coloring
        gdf.plot(column=year, cmap=cmap, linewidth=0.2, ax=ax, edgecolor="0.5", legend=False)

        # Remove axis labels and boundary box
        ax.set_axis_off()

        # Add title
        ax.set_title(year, fontsize=10)

    # Create a common colorbar for the last column
    cax = fig.add_axes([0.95, 0.15, 0.02, 0.7])  # Adjust the position and size as needed
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=gdf[years].min().min(), vmax=gdf[years].max().max()))
    sm._A = []  # fake up the array of the scalar mappable
    cbar = fig.colorbar(sm, cax=cax, orientation="vertical", format="%d", drawedges=False)  # Remove decimals
    cbar.set_label(colorbar_title, labelpad=15)

    # Hide axes for the last row
    for ax in axes[-1, :-2]:
        ax.axis('off')

    # Remove the empty subplot in the last row
    fig.delaxes(axes[-1, -1])
    fig.delaxes(axes[-1, -2])

    # Save or display the plot
    if filename:
        plt.savefig(filename, format="png", bbox_inches="tight")
    else:
        plt.show()

# Example usage
# plot_geopandas_grid(gdf, years, figsize=(10, 10), cmap="Reds", colorbar_title="Colorbar Title", filename="output_plot.svg")
# or
# plot_geopandas_grid(gdf, years, figsize=(10, 10), cmap="Reds", colorbar_title="Colorbar Title")


In [14]:
def plot_geopandas_grid(gdf, years, figsize=(12, 12), cmap="Reds", colorbar_title="Colorbar Title", 
                        boundary_shp_file="/gypsum/eguide/projects/ce8760/locations/rwanda_boundary/RWA_adm0.shp", filename=None):
    """
    This function plots a geopandas GeoDataFrame on a 3x3 grid, showing the distribution of attributes for the specified years.

    Parameters:
        - gdf: GeoPandas GeoDataFrame with columns ['administrative_boundary', '2012', '2013', ..., '2020', 'geometry'].
        - years: List of years to be plotted (e.g., ['2013', '2014', ..., '2019']).
        - figsize: Tuple specifying the figure size (default is (15, 15)).
        - cmap: Colormap to be used for coloring (default is "Reds").
        - colorbar_title: Title for the colorbar.
        - boundary_shp_file: Path to the shapefile containing the boundary.
        - filename: Optional filename to save the plot as an SVG file.

    Returns:
        - None
    """
    # Load the boundary shapefile and reproject to EPSG:4326
    boundary_gdf = gpd.read_file(boundary_shp_file).to_crs("EPSG:4326")
    
    # Reproject the main GeoDataFrame to EPSG:4326
    gdf = gdf.to_crs("EPSG:4326")

    # Set the style using seaborn
    sns.set_theme(style="ticks", palette="pastel")
    sns.set(font_scale=0.7, rc={"figure.dpi": 300, 'savefig.dpi': 300})
    # sns.set(font="Verdana", font_scale=0.7, rc={"figure.dpi": 300, 'savefig.dpi': 300})

    # Create the subplots
    fig, axes = plt.subplots(3, 3, figsize=figsize, sharex=True, sharey=True)
    
    # Plot each year on the grid
    for i, year in enumerate(years):
        row, col = divmod(i, 3)
        ax = axes[row, col]

        # Plot the boundary line (adjust linewidth and color)
        boundary_gdf.boundary.plot(ax=ax, linewidth=0.5, color="black")

        # Plot the GeoDataFrame with the specified column for coloring
        gdf.plot(column=year, cmap=cmap, linewidth=0.2, ax=ax, edgecolor="0.5", legend=False)

        # Remove axis labels and boundary box
        ax.set_axis_off()

        # Add title
        ax.set_title(year, fontsize=10)

    # Create a common colorbar for the last column
    cax = fig.add_axes([0.95, 0.15, 0.02, 0.7])  # Adjust the position and size as needed
    vmin, vmax = gdf[years].min().min(), gdf[years].max().max()
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm._A = []  # fake up the array of the scalar mappable
    cbar = fig.colorbar(sm, cax=cax, orientation="vertical", format="%d", drawedges=False)  # Remove decimals
    cbar.set_label(colorbar_title, labelpad=15)

    # Set colorbar ticks
    cbar.set_ticks([vmin, (vmin + vmax) / 2, vmax])

    # Hide axes for the last row
    for ax in axes[-1, :-2]:
        ax.axis('off')

    # Remove the empty subplot in the last row
    fig.delaxes(axes[-1, -1])
    fig.delaxes(axes[-1, -2])

    # Save or display the plot
    if filename:
        plt.savefig(filename, format="png", bbox_inches="tight")
    else:
        plt.show()

# Example usage
# plot_geopandas_grid(gdf, years, figsize=(10, 10), cmap="Reds", colorbar_title="Colorbar Title", filename="output_plot.svg")
# or
# plot_geopandas_grid(gdf, years, figsize=(10, 10), cmap="Reds", colorbar_title="Colorbar Title")
""

''

## Helper functions for calculating distance to amenities

In [15]:
import pandas as pd
from scipy.spatial import cKDTree
from shapely.geometry import Point

def calculate_min_distances(combined_data, marketplace_geojson):
    # Extract relevant columns from combined_data
    meters_data = combined_data[['meter_serial_number', 'geometry', 'transaction_date']].copy()

    # Extract relevant columns from marketplace_geojson
    marketplaces = marketplace_geojson[['id', 'geometry']].copy()

    # Convert polygon geometries to centroids
    marketplaces['geometry'] = marketplaces['geometry'].apply(lambda geom: geom.centroid if geom.type == 'Polygon' else geom)

    # Create KD-Tree for marketplaces
    marketplace_tree = cKDTree(marketplaces['geometry'].apply(lambda geom: (geom.x, geom.y)).tolist())

    # Calculate all distances for each meter
    meter_coords = meters_data['geometry'].apply(lambda geom: (geom.x, geom.y)).tolist()
    _, indices = marketplace_tree.query(meter_coords, k=1)
    distances = marketplaces.loc[indices, 'geometry'].apply(lambda geom: geom.distance(Point(coord)) for coord in meter_coords)

    # Create a dataframe to store meter_serial_number, geometry, and distance_to_marketplace
    distances_df = meters_data[['meter_serial_number', 'geometry']].copy()

    # Create columns for each month in the transaction date
    months = meters_data['transaction_date'].dt.strftime('%Y-%m').unique()
    distances_df = pd.concat([distances_df] + [pd.Series(name=month) for month in months], axis=1)

    # Update distances_df with the calculated distances
    distances_df[months] = distances.values.reshape((-1, 1))

    return distances_df

# # Example usage
# combined_data = pd.read_pickle('path/to/combined_data.pkl')
# marketplace_geojson = gpd.read_file('path/to/marketplace.geojson')
# result_df = calculate_min_distances(combined_data, marketplace_geojson)
# result_df.to_csv('path/to/output.csv', index=False)


## Getting all datasets needed

### Prepaid electricity transactions as consumption data

### Atlas AI asset wealth

In [16]:
# Asset Wealth

asset_wealth_2016_shp = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/atlas_ai_files/Rwanda_Asset-Wealth-1912m_0_08_RWA_2016.tif")
asset_wealth_2017_shp = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/atlas_ai_files/Rwanda_Asset-Wealth-1912m_0_08_RWA_2017.tif")
asset_wealth_2018_shp = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/atlas_ai_files/Rwanda_Asset-Wealth-1912m_0_08_RWA_2018.tif")
asset_wealth_2019_shp = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/atlas_ai_files/Rwanda_Asset-Wealth-1912m_0_08_RWA_2019.tif")
asset_wealth_2020_shp = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/atlas_ai_files/Rwanda_Asset-Wealth-1912m_0_08_RWA_2020.tif")

asset_wealth_2016_shp = asset_wealth_2016_shp[asset_wealth_2016_shp["pixel_value"]<3]
asset_wealth_2017_shp = asset_wealth_2017_shp[asset_wealth_2017_shp["pixel_value"]<3]
asset_wealth_2018_shp = asset_wealth_2018_shp[asset_wealth_2018_shp["pixel_value"]<3]
asset_wealth_2019_shp = asset_wealth_2019_shp[asset_wealth_2019_shp["pixel_value"]<3]
asset_wealth_2020_shp = asset_wealth_2020_shp[asset_wealth_2020_shp["pixel_value"]<3]

village_asset_wealth_2016 = compute_administrative_metric(asset_wealth_2016_shp, "village")
village_asset_wealth_2017 = compute_administrative_metric(asset_wealth_2017_shp, "village")
village_asset_wealth_2018 = compute_administrative_metric(asset_wealth_2018_shp, "village")
village_asset_wealth_2019 = compute_administrative_metric(asset_wealth_2019_shp, "village")
village_asset_wealth_2020 = compute_administrative_metric(asset_wealth_2020_shp, "village")

### Atlas AI spending

In [17]:
# Spending

spending_2016_shp = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/atlas_ai_files/Rwanda_Spending-1912m_0_12_RWA_2016.tif")
spending_2017_shp = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/atlas_ai_files/Rwanda_Spending-1912m_0_12_RWA_2017.tif")
spending_2018_shp = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/atlas_ai_files/Rwanda_Spending-1912m_0_12_RWA_2018.tif")
spending_2019_shp = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/atlas_ai_files/Rwanda_Spending-1912m_0_12_RWA_2019.tif")
spending_2020_shp = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/atlas_ai_files/Rwanda_Spending-1912m_0_12_RWA_2020.tif")

spending_2016_shp = spending_2016_shp[spending_2016_shp["pixel_value"]<1000]
spending_2017_shp = spending_2017_shp[spending_2017_shp["pixel_value"]<1000]
spending_2018_shp = spending_2018_shp[spending_2018_shp["pixel_value"]<1000]
spending_2019_shp = spending_2019_shp[spending_2019_shp["pixel_value"]<1000]
spending_2020_shp = spending_2020_shp[spending_2020_shp["pixel_value"]<1000]

village_spending_2016 = compute_administrative_metric(spending_2016_shp, "village")
village_spending_2017 = compute_administrative_metric(spending_2017_shp, "village")
village_spending_2018 = compute_administrative_metric(spending_2018_shp, "village")
village_spending_2019 = compute_administrative_metric(spending_2019_shp, "village")
village_spending_2020 = compute_administrative_metric(spending_2020_shp, "village")

### Degree of Urbanization

In [18]:
# Degree of Urbanization

urban_2010_shp = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/european_commision/GHS_SMOD_E2010_GLOBE_R2023A_54009_1000_V1_0.tif")
urban_2015_shp = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/european_commision/GHS_SMOD_E2015_GLOBE_R2023A_54009_1000_V1_0.tif")
urban_2020_shp = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/european_commision/GHS_SMOD_E2020_GLOBE_R2023A_54009_1000_V1_0.tif")

urban_2010_shp = urban_2010_shp[urban_2010_shp["pixel_value"]>0]
urban_2015_shp = urban_2015_shp[urban_2015_shp["pixel_value"]>0]
urban_2020_shp = urban_2020_shp[urban_2020_shp["pixel_value"]>0]

village_urban_2010 = compute_administrative_metric(urban_2010_shp, "village", summary_method="mode")
village_urban_2015 = compute_administrative_metric(urban_2015_shp, "village", summary_method="mode")
village_urban_2020 = compute_administrative_metric(urban_2020_shp, "village", summary_method="mode")

### Building heights

In [19]:
building_height_2018 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/european_commision/GHS_built_H_2018.tif")
building_height_2018 = building_height_2018[building_height_2018["pixel_value"]>0]

village_building_height_2018 = compute_administrative_metric(building_height_2018, "village")

### Building Volume

In [20]:
# building_volume_2010 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/european_commision/GHS_Built_V_2010.tif")
# building_volume_2015 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/european_commision/GHS_Built_V_2015.tif")
# building_volume_2020 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/european_commision/GHS_Built_V_2020.tif")


In [21]:
# village_building_volume_2010 = compute_administrative_metric(building_volume_2010, "village")
# village_building_volume_2015 = compute_administrative_metric(building_volume_2015, "village")
# village_building_volume_2020 = compute_administrative_metric(building_volume_2020, "village")

In [22]:
# village_building_volume_2010.to_pickle("/gypsum/eguide/projects/ce8760/european_commision/village_building_volume_2010.pkl")
# village_building_volume_2015.to_pickle("/gypsum/eguide/projects/ce8760/european_commision/village_building_volume_2015.pkl")
# village_building_volume_2020.to_pickle("/gypsum/eguide/projects/ce8760/european_commision/village_building_volume_2020.pkl")

# How to read
# a = pd.read_pickle("/gypsum/eguide/projects/ce8760/european_commision/village_building_volume_2020.pkl")

In [23]:
village_building_volume_2010 = pd.read_pickle("/gypsum/eguide/projects/ce8760/european_commision/village_building_volume_2010.pkl")
village_building_volume_2015 = pd.read_pickle("/gypsum/eguide/projects/ce8760/european_commision/village_building_volume_2015.pkl")
village_building_volume_2020 = pd.read_pickle("/gypsum/eguide/projects/ce8760/european_commision/village_building_volume_2020.pkl")

### Population

In [24]:
# population_grid_2010 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/Rwanda_100m_Population/RWA_pph_2010_adj_v2.tif")
# population_grid_2015 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/Rwanda_100m_Population/RWA_pph_2015_adj_v2.tif")
# population_grid_2020 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/Rwanda_100m_Population/RWA_pph_2020_adj_v2.tif")

In [25]:
# village_population_grid_2010 = compute_administrative_metric(population_grid_2010, "village")
# village_population_grid_2015 = compute_administrative_metric(population_grid_2015, "village")
# village_population_grid_2020 = compute_administrative_metric(population_grid_2020, "village")

In [26]:
# # population
# population_2012 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/rwanda_population/rwa_ppp_2012.tif")
# population_2013 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/rwanda_population/rwa_ppp_2013.tif")
# population_2014 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/rwanda_population/rwa_ppp_2014.tif")
# population_2015 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/rwanda_population/rwa_ppp_2015.tif")
# population_2016 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/rwanda_population/rwa_ppp_2016.tif")
# population_2017 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/rwanda_population/rwa_ppp_2017.tif")
# population_2018 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/rwanda_population/rwa_ppp_2018.tif")
# population_2019 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/rwanda_population/rwa_ppp_2019.tif")
# population_2020 = parallel_clip_large_tif("/gypsum/eguide/projects/ce8760/rwanda_population/rwa_ppp_2020.tif")

In [27]:
# village_population_2012 = compute_administrative_metric(population_2012, "village")
# village_population_2013 = compute_administrative_metric(population_2013, "village")
# village_population_2014 = compute_administrative_metric(population_2014, "village")
# village_population_2015 = compute_administrative_metric(population_2015, "village")
# village_population_2016 = compute_administrative_metric(population_2016, "village")
# village_population_2017 = compute_administrative_metric(population_2017, "village")
# village_population_2018 = compute_administrative_metric(population_2018, "village")
# village_population_2019 = compute_administrative_metric(population_2019, "village")
# village_population_2020 = compute_administrative_metric(population_2020, "village")

In [28]:
# village_population_2012.to_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2012.pkl")
# village_population_2013.to_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2013.pkl")
# village_population_2014.to_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2014.pkl")
# village_population_2015.to_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2015.pkl")
# village_population_2016.to_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2016.pkl")
# village_population_2017.to_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2017.pkl")
# village_population_2018.to_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2018.pkl")
# village_population_2019.to_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2019.pkl")
# village_population_2020.to_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2020.pkl")


village_population_2012 = pd.read_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2012.pkl")
village_population_2013 = pd.read_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2013.pkl")
village_population_2014 = pd.read_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2014.pkl")
village_population_2015 = pd.read_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2015.pkl")
village_population_2016 = pd.read_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2016.pkl")
village_population_2017 = pd.read_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2017.pkl")
village_population_2018 = pd.read_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2018.pkl")
village_population_2019 = pd.read_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2019.pkl")
village_population_2020 = pd.read_pickle("/gypsum/eguide/projects/ce8760/population/village_population_2020.pkl")

In [29]:
# core_refactor.py

import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.spatial import cKDTree
from geopy.distance import geodesic
import os

def load_combined_data(pickle_path, connection_type):
    df = pd.read_pickle(pickle_path)
    df = df[df["connection_type"] == connection_type]
    df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')
    df['installation_date'] = pd.to_datetime(df['installation_date'], errors='coerce')
    return df

def preprocess_meter_locations(df):
    grouped = df.groupby([
        df["meter_serial_number"],
        df["connection_type"],
        df['transaction_date'].dt.year
    ]).agg({
        "geometry": "first",
        "sector_id": "first",
        "cell_id": "first",
        "village_id": "first"
    }).reset_index()
    return grouped.groupby("meter_serial_number")[["geometry"]].first().reset_index(), grouped

def extract_coords(geom):
    if geom.geom_type == 'Polygon':
        return geom.centroid.coords[0]
    elif geom.geom_type == 'Point':
        return geom.coords[0]
    elif geom.geom_type == 'MultiPolygon':
        largest_polygon = max(geom.geoms, key=lambda p: p.area)
        return largest_polygon.centroid.coords[0]
    elif geom.geom_type == 'MultiPoint':
        return geom.geoms[0].coords[0]
    else:
        raise ValueError(f"Unsupported geometry type: {geom.geom_type}")

def build_ckdtree_from_geoms(gdf):
    coords = gdf['geometry'].apply(extract_coords)
    return cKDTree(list(coords))

def compute_nearest_distances(src_points, tree):
    src_coords = np.array([[pt.x, pt.y] for pt in src_points])
    distances, _ = tree.query(src_coords, k=1)
    return [geodesic((0, 0), (0, d)).kilometers for d in distances]

def create_distance_summary(df, meter_df, column_name):
    df = pd.merge(df, meter_df[["meter_serial_number", column_name]], on="meter_serial_number", how="left")
    return df.groupby(["village_id", "transaction_date"])[column_name].median().reset_index()

def extract_min_distance_column(df, tree, column_name):
    def extract(row):
        point = row['geometry'].coords[0]
        distance, _ = tree.query(point)
        return geodesic((0, 0), (0, distance)).kilometers
    df[column_name] = df.apply(extract, axis=1)
    return df

def extract_tariffs_or_consumption(path, column_name):
    df = pd.read_csv(path).dropna()
    df = df.melt(id_vars="administrative_id", var_name="transaction_date", value_name=column_name)
    df.columns = ["village_id", "date", column_name]
    df = df[~df["date"].isin([2012, 2020, "geometry"])]
    return df.reset_index(drop=True).iloc[:-1]

def join_annual_tables(table_list, column_name):
    combined = []
    for df in table_list:
        year = df["date"].iloc[0]
        temp = df.rename(columns={
            "Village_ID": "village_id",
            "pixel_value": column_name
        })
        temp["year"] = str(year)
        temp = temp[["village_id", column_name, "year"]]
        combined.append(temp)
    return pd.concat(combined, ignore_index=True)

def merge_features(final_df, *features):
    for i, feature_df in enumerate(features):
        if "village_id" not in feature_df.columns or "year" not in feature_df.columns:
            print(f"❌ Feature at position {i} is missing 'village_id' or 'year'. Columns: {feature_df.columns.tolist()}")
            continue  # or raise an error if preferred
        final_df = pd.merge(final_df, feature_df, on=["village_id", "year"], how="outer")
    return final_df

def enrich_with_static_features(final_df,
                                 asset_list,
                                 spending_list,
                                 population_list,
                                 urbanization_list,
                                 building_volume_list,
                                 building_height,
                                 landcover_path,
                                 landcover_yearly_paths):
    asset_df = join_annual_tables(asset_list, "asset_wealth")
    spending_df = join_annual_tables(spending_list, "spending")
    population_df = join_annual_tables(population_list, "population")
    urbanization_df = join_annual_tables(urbanization_list, "urbanization")
    building_volume_df = join_annual_tables(building_volume_list, "building_volume")

    building_height = building_height.drop(columns=["geometry"])
    building_height.columns = ["village_id", "building_height"]
    building_height["year"] = "2018"

    landcover = pd.read_csv(landcover_path)
    landcover["year"] = landcover["year"].astype(str)
    landcover["village_id"] = landcover["village_id"].astype(str)
    yearly_dfs = [pd.read_pickle(p) for p in landcover_yearly_paths]
    for df in yearly_dfs:
        df.columns = landcover.columns
    landcover = pd.concat(yearly_dfs + [landcover])
    landcover["year"] = landcover["year"].astype("str")
    landcover["village_id"] = landcover["village_id"].astype("str")

    final_df["year"] = final_df["year"].astype("str")
    final_df["village_id"] = final_df["village_id"].astype("str")

    return merge_features(final_df, asset_df, spending_df, population_df,
                          urbanization_df, building_volume_df, building_height, landcover)

def generate_final_df(connection_type, base_path):
    combined_data = load_combined_data(os.path.join(base_path, "combined_data.pkl"), connection_type)
    meter_location, test_combined = preprocess_meter_locations(combined_data)

    rwa_roads = gpd.read_file(os.path.join(base_path, "rwa_road/rwa_road.shp"))
    rwa_roads = rwa_roads.to_crs("EPSG:4326")
    rwa_roads = rwa_roads[rwa_roads["type"].isin(["Primary Route", "Secondary Route"])]

    road_points = []
    for road in rwa_roads.geometry:
        road_points.extend([road.interpolate(i / 1000, normalized=True) for i in range(1001)])

    road_tree = cKDTree([[pt.x, pt.y] for pt in road_points])
    meter_array = np.array([[pt.x, pt.y] for pt in meter_location.geometry])
    distances, _ = road_tree.query(meter_array, k=1)
    meter_location['distance_to_nearest_road'] = [geodesic((0, 0), (0, d)).km for d in distances]

    test_combined = pd.merge(test_combined, meter_location[['meter_serial_number', 'distance_to_nearest_road']],
                             on='meter_serial_number', how='left')

    village_distance_to_nearest_road = test_combined.groupby(["village_id", "transaction_date"])["distance_to_nearest_road"].median().reset_index()

    amenities = {
        "market": "open_street_map/marketplace.geojson",
        "busstation": "open_street_map/bus_station.geojson",
        "schools": "open_street_map/school_export.geojson",
        "banks": "open_street_map/banks.geojson"
    }

    for key, path in amenities.items():
        gdf = gpd.read_file(os.path.join(base_path, path)).to_crs("EPSG:4326")
        tree = build_ckdtree_from_geoms(gdf)
        test_combined = extract_min_distance_column(test_combined, tree, f"distance_{key}")

    distance_tables = {
        "distance_market": "distance_market",
        "distance_busstation": "distance_busstation",
        "distance_schools": "distance_schools",
        "distance_banks": "distance_banks"
    }

    final_df = village_distance_to_nearest_road.copy()
    for col in distance_tables:
        df = test_combined.groupby(["village_id", "transaction_date"])[col].median().reset_index()
        final_df = pd.merge(final_df, df, on=["village_id", "transaction_date"], how="outer")

    final_df = final_df.rename(columns={"transaction_date": "year"})
    final_df["year"] = final_df["year"].astype(str)

    return final_df


In [30]:
res_final_df = generate_final_df(connection_type="Residential", base_path="/gypsum/eguide/projects/ce8760")

In [31]:
non_res_final_df = generate_final_df(connection_type="Non Residential", base_path="/gypsum/eguide/projects/ce8760")

Enrich with other variables

In [32]:
non_res_final_df

,village_id,year,distance_to_nearest_road,distance_market,distance_busstation,distance_schools,distance_banks
0,11010102,2015,0.248883,1.064686,0.713934,0.266333,0.429359
1,11010102,2016,0.248883,1.064686,0.713934,0.266333,0.429359
2,11010102,2017,0.248883,1.064686,0.713934,0.266333,0.429359
3,11010102,2018,0.248883,1.064686,0.713934,0.266333,0.429359
4,11010102,2019,0.248883,1.064686,0.713934,0.266333,0.429359
...,...,...,...,...,...,...,...
18074,57150504,2017,2.672471,15.623530,17.417366,3.310920,17.556683
18075,57150504,2018,2.699360,15.624092,17.405460,3.317234,17.544696
18076,57150504,2019,2.726248,15.623530,17.393553,3.323547,17.532708
18077,57150504,2020,2.726248,15.620868,17.393553,3.323681,17.532708


In [33]:
# === Map years to corresponding dataframes ===

asset_dfs = [village_asset_wealth_2016, village_asset_wealth_2017, village_asset_wealth_2018, village_asset_wealth_2019, village_asset_wealth_2020]
spending_dfs = [village_spending_2016, village_spending_2017, village_spending_2018, village_spending_2019, village_spending_2020]
population_dfs = [village_population_2012, village_population_2013, village_population_2014,
                  village_population_2015, village_population_2016, village_population_2017,
                  village_population_2018, village_population_2019, village_population_2020]
urbanization_dfs = [village_urban_2010, village_urban_2015, village_urban_2020]
building_volume_dfs = [village_building_volume_2010, village_building_volume_2015, village_building_volume_2020]

asset_years = ["2016", "2017", "2018", "2019", "2020"]
spending_years = ["2016", "2017", "2018", "2019", "2020"]
population_years = ["2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]
urbanization_years = ["2010", "2015", "2020"]
building_volume_years = ["2010", "2015", "2020"]

# === Assign 'date' column ===

def assign_year(dfs, years):
    for df, yr in zip(dfs, years):
        df["date"] = yr

assign_year(asset_dfs, asset_years)
assign_year(spending_dfs, spending_years)
assign_year(population_dfs, population_years)
assign_year(urbanization_dfs, urbanization_years)
assign_year(building_volume_dfs, building_volume_years)

In [34]:
res_final_df = enrich_with_static_features(
    res_final_df,
    asset_list=asset_dfs,
    spending_list=spending_dfs,
    population_list=population_dfs,
    urbanization_list=urbanization_dfs,
    building_volume_list=building_volume_dfs,
    building_height=village_building_height_2018,
    landcover_path="/gypsum/eguide/projects/ce8760/data/landcover/landcover.csv",
    landcover_yearly_paths=[
        "/gypsum/eguide/projects/ce8760/data/landcover/df_2013.pkl",
        "/gypsum/eguide/projects/ce8760/data/landcover/df_2014.pkl",
        "/gypsum/eguide/projects/ce8760/data/landcover/df_2015.pkl",
        "/gypsum/eguide/projects/ce8760/data/landcover/df_2016.pkl"
    ]
)

In [35]:
non_res_final_df = enrich_with_static_features(
    non_res_final_df,
    asset_list=asset_dfs,
    spending_list=spending_dfs,
    population_list=population_dfs,
    urbanization_list=urbanization_dfs,
    building_volume_list=building_volume_dfs,
    building_height=village_building_height_2018,
    landcover_path="/gypsum/eguide/projects/ce8760/data/landcover/landcover.csv",
    landcover_yearly_paths=[
        "/gypsum/eguide/projects/ce8760/data/landcover/df_2013.pkl",
        "/gypsum/eguide/projects/ce8760/data/landcover/df_2014.pkl",
        "/gypsum/eguide/projects/ce8760/data/landcover/df_2015.pkl",
        "/gypsum/eguide/projects/ce8760/data/landcover/df_2016.pkl"
    ]
)

In [36]:
res_final_df.to_csv("/gypsum/eguide/projects/ce8760/res_final_df.csv", index=False)
non_res_final_df.to_csv("/gypsum/eguide/projects/ce8760/non_res_final_df.csv", index=False)

In [37]:
res_final_df

,village_id,year,distance_to_nearest_road,distance_market,distance_busstation,distance_schools,distance_banks,asset_wealth,spending,population,urbanization,building_volume,building_height,cropland_proportion,builtarea_proportion,rangeland_proportion
0,11010102,2012,0.248803,0.988930,0.765208,0.172175,0.443753,NaN,NaN,256.252640,NaN,NaN,NaN,NaN,NaN,NaN
1,11010102,2013,0.248803,0.988930,0.767398,0.172175,0.439669,NaN,NaN,257.667900,NaN,NaN,NaN,0.000000,1.000000,0.000000
2,11010102,2014,0.249891,0.988848,0.765208,0.173555,0.438713,NaN,NaN,265.239548,NaN,NaN,NaN,0.000000,1.000000,0.000000
3,11010102,2015,0.249891,0.988191,0.765208,0.173555,0.442250,NaN,NaN,251.027534,30.0,50853.0,NaN,0.000000,1.000000,0.000000
4,11010102,2016,0.249891,0.988191,0.765208,0.173555,0.442250,1.070581,6.413167,256.711594,NaN,NaN,NaN,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177775,57150501,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300282,0.158434,0.007784
177776,57150502,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.214361,0.196575,0.150272
177777,57150503,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.209166,0.133858,0.177291
177778,57150504,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.067908,0.430303,0.050098
